In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as ddf
from pandas import Series, DataFrame
from shapely import wkt
import datetime
import math

### Upload the county boundary data we just saved

In [2]:
ur_files = ddf.read_csv(r'county_boundary.csv', dtype = {'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'NAME': str,'NAMELSAD': str})
county = ur_files.compute()
county

,Unnamed: 0,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17..."
1,1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4..."
2,2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9..."
3,3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999..."
4,4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4..."
...,...,...,...,...,...,...,...,...,...,...,...
3228,3228,31,073,835858,0500000US31073,31073,Gosper,6,1186616237,11831826,"POLYGON ((-100.095095 40.438656, -100.089374 4..."
3229,3229,39,075,1074050,0500000US39075,39075,Holmes,6,1094405866,3695230,"POLYGON ((-82.220665 40.66758, -82.193271 40.6..."
3230,3230,48,171,1383871,0500000US48171,48171,Gillespie,6,2740719114,9012764,"POLYGON ((-99.303996 30.499831999999998, -99.2..."
3231,3231,55,079,1581100,0500000US55079,55079,Milwaukee,6,625440563,2455383635,"POLYGON ((-88.069586 42.867263, -88.069586 42...."


In [3]:
# get rid of the 'Unnamed column'
county = county.loc[:, ~county.columns.str.contains('^Unnamed')]
county.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17..."
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4..."
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9..."
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999..."
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4..."


### Find the centroid for each county based on the geometry column

In [4]:
def findCentroid(g):
    p1 = wkt.loads(g)
    return p1.centroid.wkt

In [5]:
county['Centroid'] = county['geometry'].apply(findCentroid)

In [6]:
county

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,Centroid
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",POINT (-88.99925588833919 37.05848196169251)
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",POINT (-84.21715057392247 38.20673452494952)
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",POINT (-86.68162334315123 37.20728450321774)
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",POINT (-83.96431121402227 37.69244422727429)
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",POINT (-83.69665601093851 38.37011818128263)
...,...,...,...,...,...,...,...,...,...,...,...
3228,31,073,835858,0500000US31073,31073,Gosper,6,1186616237,11831826,"POLYGON ((-100.095095 40.438656, -100.089374 4...",POINT (-99.83070272142194 40.51481873658771)
3229,39,075,1074050,0500000US39075,39075,Holmes,6,1094405866,3695230,"POLYGON ((-82.220665 40.66758, -82.193271 40.6...",POINT (-81.92934087401707 40.56120787095739)
3230,48,171,1383871,0500000US48171,48171,Gillespie,6,2740719114,9012764,"POLYGON ((-99.303996 30.499831999999998, -99.2...",POINT (-98.94655484362227 30.31803705160264)
3231,55,079,1581100,0500000US55079,55079,Milwaukee,6,625440563,2455383635,"POLYGON ((-88.069586 42.867263, -88.069586 42....",POINT (-87.96677179509463 43.00719656824836)


In [7]:
county['Centroid'].dtype

dtype('O')

### Upload the climate data, using 2020 daily climate data as an example

In [8]:
ur_files=ddf.read_csv(r'Daily_Climate/DailyTemp_full_2020.csv', dtype={'STATION': 'object','FRSHTT': 'object'})
climate = ur_files.compute()
climate = climate.loc[:, ~climate.columns.str.contains('^Unnamed')]
climate

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,6,73.2,6,...,21.0,83.3,*,82.2,*,0.00,I,NaN,0,2020
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,8,72.4,8,...,17.1,83.7,*,82.4,*,0.00,I,NaN,0,2020
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,24,29.3,24,...,24.1,35.8,,23.5,*,0.37,G,NaN,10000,2020
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,23.4,24,...,49.1,34.2,*,19.0,,0.54,E,NaN,101000,2020
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,21,18.9,21,...,65.9,33.6,*,19.6,*,0.31,E,NaN,10000,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308054,A5125600451,2020-12-27,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",54.5,24,33.5,24,...,32.1,62.6,*,48.2,*,0.00,I,NaN,0,2020
308055,A5125600451,2020-12-28,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",40.4,24,30.2,24,...,17.1,55.4,*,30.2,*,0.00,I,NaN,0,2020
308056,A5125600451,2020-12-29,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",34.9,24,31.4,24,...,25.1,39.2,*,32.0,*,NaN,,NaN,11000,2020
308057,A5125600451,2020-12-30,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",41.5,24,41.5,24,...,21.0,51.8,*,32.0,*,NaN,,NaN,111000,2020


### Rename the columns based the data description from NOAA

In [9]:
def renameColumns(df):
    df = df.rename(columns = {'STATION':'station_id','NAME':'station_name',
                            'LATITUDE':'station_lat', 'LONGITUDE': 'station_lon',
                            'ELEVATION': 'elevation','DATE': 'date', 'TEMP':
                            'mean_temp', 'MIN': 'min_temp', 'MAX': 'max_temp',
                            'DEWP':'dewpoint', 'SLP': 'sea_level_pressure', 'STP':
                            'station_pressure', 'VISIB': 'visibility', 'WDSP':
                            'wind_speed', 'MXSPD': 'max_wind_speed', 'GUST':
                            'wind_gust', 'PRCP': 'precipitation','PRCP_ATTRIBUTES':
                            'precip_flag', 'YEAR':'year'})
    return df

In [10]:
climate = renameColumns(climate)
climate.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,TEMP_ATTRIBUTES,dewpoint,DEWP_ATTRIBUTES,...,wind_gust,max_temp,MAX_ATTRIBUTES,min_temp,MIN_ATTRIBUTES,precipitation,precip_flag,SNDP,FRSHTT,year
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,6,73.2,6,...,21.0,83.3,*,82.2,*,0.00,I,NaN,0,2020
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,8,72.4,8,...,17.1,83.7,*,82.4,*,0.00,I,NaN,0,2020
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,24,29.3,24,...,24.1,35.8,,23.5,*,0.37,G,NaN,10000,2020
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,23.4,24,...,49.1,34.2,*,19.0,,0.54,E,NaN,101000,2020
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,21,18.9,21,...,65.9,33.6,*,19.6,*,0.31,E,NaN,10000,2020


In [12]:
len(climate['station_id'].unique())

12299

In [13]:
climate.columns

Index(['station_id', 'date', 'station_lat', 'station_lon', 'elevation',
       'station_name', 'mean_temp', 'TEMP_ATTRIBUTES', 'dewpoint',
       'DEWP_ATTRIBUTES', 'sea_level_pressure', 'SLP_ATTRIBUTES',
       'station_pressure', 'STP_ATTRIBUTES', 'visibility', 'VISIB_ATTRIBUTES',
       'wind_speed', 'WDSP_ATTRIBUTES', 'max_wind_speed', 'wind_gust',
       'max_temp', 'MAX_ATTRIBUTES', 'min_temp', 'MIN_ATTRIBUTES',
       'precipitation', 'precip_flag', 'SNDP', 'FRSHTT', 'year'],
      dtype='object')

#### Find the month information and convert FRSHTT column

In [14]:
def getMonth(date):
    return(date.strftime('%b'))

In [15]:
climate['date'] = pd.to_datetime(climate['date'])
climate['Month'] = climate['date'].apply(getMonth)

##### 'FRSHTT' is a 6 digit string representing Fog, Rain, Snow, Hail, Thunder, Tornado. '1' means the weather event occurs on that day, '0' mean not occurs. So we need to seprent 'FRSHTT' string to find the real weather information

In [16]:
def convertString(f):
    return f.rjust(6, '0')

In [17]:
climate['FRSHTT'] = climate['FRSHTT'].apply(convertString)
climate.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,TEMP_ATTRIBUTES,dewpoint,DEWP_ATTRIBUTES,...,max_temp,MAX_ATTRIBUTES,min_temp,MIN_ATTRIBUTES,precipitation,precip_flag,SNDP,FRSHTT,year,Month
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,6,73.2,6,...,83.3,*,82.2,*,0.00,I,NaN,000000,2020,Mar
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,8,72.4,8,...,83.7,*,82.4,*,0.00,I,NaN,000000,2020,Mar
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,24,29.3,24,...,35.8,,23.5,*,0.37,G,NaN,010000,2020,Jan
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,23.4,24,...,34.2,*,19.0,,0.54,E,NaN,101000,2020,Jan
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,21,18.9,21,...,33.6,*,19.6,*,0.31,E,NaN,010000,2020,Jan


In [18]:
def findFog(F):
    return int(F[0])

def findRain(F):
    return int(F[1])

def findSnow(F):
    return int(F[2])

def findHail(F):
    return int(F[3])

def findThunder(F):
    return int(F[4])

def findTornado(F):
    return int(F[5])

In [19]:
def seperateFRSHTT(df):
    df['fog'] = df['FRSHTT'].apply(findFog)
    df['rain'] = df['FRSHTT'].apply(findRain)
    df['snow'] = df['FRSHTT'].apply(findSnow)
    df['hail'] = df['FRSHTT'].apply(findHail)
    df['thunder'] = df['FRSHTT'].apply(findThunder)
    df['tornado'] = df['FRSHTT'].apply(findTornado)
    return df

In [20]:
climate = seperateFRSHTT(climate)
climate.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,TEMP_ATTRIBUTES,dewpoint,DEWP_ATTRIBUTES,...,SNDP,FRSHTT,year,Month,fog,rain,snow,hail,thunder,tornado
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,6,73.2,6,...,NaN,000000,2020,Mar,0,0,0,0,0,0
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,8,72.4,8,...,NaN,000000,2020,Mar,0,0,0,0,0,0
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,24,29.3,24,...,NaN,010000,2020,Jan,0,1,0,0,0,0
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,23.4,24,...,NaN,101000,2020,Jan,1,0,1,0,0,0
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,21,18.9,21,...,NaN,010000,2020,Jan,0,1,0,0,0,0


### Select the columns we need

In [21]:
def selectColumns(df):
    df = df[['station_id', 'date', 'station_lat', 'station_lon', 'elevation','station_name', 'mean_temp',
             'min_temp', 'max_temp', 'dewpoint', 'sea_level_pressure', 'station_pressure', 'visibility', 
             'wind_speed','precipitation', 'fog', 'rain', 'snow','hail', 'thunder', 'tornado','year', 'Month']]
    return df
    return df

In [22]:
climate = selectColumns(climate)
climate.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,min_temp,max_temp,dewpoint,...,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado,year,Month
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,82.2,83.3,73.2,...,14.5,0.00,0,0,0,0,0,0,2020,Mar
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,82.4,83.7,72.4,...,11.0,0.00,0,0,0,0,0,0,2020,Mar
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23.5,35.8,29.3,...,9.9,0.37,0,1,0,0,0,0,2020,Jan
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,19.0,34.2,23.4,...,23.0,0.54,1,0,1,0,0,0,2020,Jan
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,19.6,33.6,18.9,...,33.4,0.31,0,1,0,0,0,0,2020,Jan


#### When we need to find the nearest weather station for each county, we need to find the yearly climate dataframe first since it will take a long time to work on a daily climate data frame. So let prepare the yearly climate dataframe first. 

while some of most of the record we can find the yearly average record, columns 'fog', 'rain', 'snow', 'hail','thunder', 'tornado' represented by '0' or 1, we should not calcuate the average values, instead we can sum the days of each weather event occured in each year. In this situation, we need to seperate the daily climate data frame to two parts. Note: the spatial and temporal information should be included in each dataframe

In [30]:
climate_part1 = climate[['station_id', 'date', 'station_lat', 'station_lon','elevation','station_name', 'mean_temp', 
                         'min_temp', 'max_temp','dewpoint', 'sea_level_pressure','station_pressure','visibility',
                         'wind_speed', 'precipitation','year', 'Month']]
climate_part2 = climate[['station_id', 'date', 'station_lat', 'station_lon','elevation','station_name','fog', 'rain', 
                         'snow', 'hail','thunder', 'tornado', 'year', 'Month']]

In [31]:
climate_part1_yearly = climate_part1.groupby(['station_id', 'station_lat', 'station_lon','elevation','station_name', 'year']).mean()
climate_part1_yearly = climate_part1_yearly.reset_index()
climate_part1_yearly.head()

,station_id,station_lat,station_lon,elevation,station_name,year,mean_temp,min_temp,max_temp,dewpoint,sea_level_pressure,station_pressure,visibility,wind_speed,precipitation
0,10000199999,51.602414,6.142172,32.30,"NIEDERRHEIN, GM",2020,54.705725,47.703448,60.793103,45.812214,NaN,NaN,5.816412,7.288550,0.000000
1,1001099999,70.933333,-8.666667,9.00,"JAN MAYEN NOR NAVY, NO",2020,32.702466,29.169589,36.341370,28.609041,1005.194490,312.542149,9.962117,14.442192,0.092577
2,1001499999,59.791925,5.340850,48.76,"SORSTOKKEN, NO",2020,49.737367,46.426277,52.768727,43.220641,NaN,NaN,5.941993,8.966904,0.000000
3,10018099999,54.913250,8.340472,15.54,"WESTERLAND SYLT, GM",2020,52.437152,48.524116,56.634615,47.586131,NaN,NaN,5.510526,12.553251,0.000000
4,1002099999,80.050000,16.250000,8.00,"VERLEGENHUKEN, NO",2020,17.036948,12.869880,21.083936,13.459438,1006.286585,298.541767,NaN,12.864659,0.000000


In [32]:
climate_part2_yearly = climate_part2.groupby(['station_id', 'station_lat', 'station_lon','elevation','station_name', 'year']).sum()
climate_part2_yearly = climate_part2_yearly.reset_index()
climate_part2_yearly.head()

,station_id,station_lat,station_lon,elevation,station_name,year,fog,rain,snow,hail,thunder,tornado
0,10000199999,51.602414,6.142172,32.30,"NIEDERRHEIN, GM",2020,15,136,2,0,4,0
1,1001099999,70.933333,-8.666667,9.00,"JAN MAYEN NOR NAVY, NO",2020,111,173,143,2,0,0
2,1001499999,59.791925,5.340850,48.76,"SORSTOKKEN, NO",2020,10,159,22,1,1,0
3,10018099999,54.913250,8.340472,15.54,"WESTERLAND SYLT, GM",2020,8,110,0,0,2,0
4,1002099999,80.050000,16.250000,8.00,"VERLEGENHUKEN, NO",2020,0,0,0,0,0,0


In [34]:
# combine these two parts to a yearly climate dataframe
climate_yearly = climate_part1_yearly.merge(climate_part2_yearly, on = ['station_id','station_lat','station_lon','elevation','station_name', 'year'], how ='left')
climate_yearly.head()

,station_id,station_lat,station_lon,elevation,station_name,year,mean_temp,min_temp,max_temp,dewpoint,...,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,10000199999,51.602414,6.142172,32.30,"NIEDERRHEIN, GM",2020,54.705725,47.703448,60.793103,45.812214,...,NaN,5.816412,7.288550,0.000000,15,136,2,0,4,0
1,1001099999,70.933333,-8.666667,9.00,"JAN MAYEN NOR NAVY, NO",2020,32.702466,29.169589,36.341370,28.609041,...,312.542149,9.962117,14.442192,0.092577,111,173,143,2,0,0
2,1001499999,59.791925,5.340850,48.76,"SORSTOKKEN, NO",2020,49.737367,46.426277,52.768727,43.220641,...,NaN,5.941993,8.966904,0.000000,10,159,22,1,1,0
3,10018099999,54.913250,8.340472,15.54,"WESTERLAND SYLT, GM",2020,52.437152,48.524116,56.634615,47.586131,...,NaN,5.510526,12.553251,0.000000,8,110,0,0,2,0
4,1002099999,80.050000,16.250000,8.00,"VERLEGENHUKEN, NO",2020,17.036948,12.869880,21.083936,13.459438,...,298.541767,NaN,12.864659,0.000000,0,0,0,0,0,0


#### From the county boundary dataframe, we need to find the longitude and latitude from the centroid information. 

In [35]:
county.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,Centroid,lat,lon
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",POINT (-88.99925588833919 37.05848196169251),37.058482,-88.999256
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",POINT (-84.21715057392247 38.20673452494952),38.206735,-84.217151
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",POINT (-86.68162334315123 37.20728450321774),37.207285,-86.681623
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",POINT (-83.96431121402227 37.69244422727429),37.692444,-83.964311
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",POINT (-83.69665601093851 38.37011818128263),38.370118,-83.696656


In [36]:
def findLon(point):
    strList = point.split(' ')
    lon = strList[1][1:]
    lon = float(lon)
    return lon

In [37]:
def findLat(point):
    strList = point.split(' ')
    lat = strList[2][:-1]
    lat = float(lat)
    return lat

In [38]:
county['lat'] = county['Centroid'].apply(findLat)
county['lon'] = county['Centroid'].apply(findLon)

In [39]:
county.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,Centroid,lat,lon
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",POINT (-88.99925588833919 37.05848196169251),37.058482,-88.999256
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",POINT (-84.21715057392247 38.20673452494952),38.206735,-84.217151
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",POINT (-86.68162334315123 37.20728450321774),37.207285,-86.681623
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",POINT (-83.96431121402227 37.69244422727429),37.692444,-83.964311
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",POINT (-83.69665601093851 38.37011818128263),38.370118,-83.696656


#### We need to create a list of weather station list from the yearly climate dataframe withing conbining the longitude and latitude information since the longitude and latitude information are used to calculate the distance from the centroid to the weather stations; the station_id should be used to identify the nearest weather station we need

In [40]:
def ID_Coordinate(ID,lat, lon):
    setList = []
    setList.append(ID)
    setList.append(lat)
    setList.append(lon)
    return setList

In [41]:
climate_yearly['ID_Coordinate'] = climate_yearly.apply(lambda x:ID_Coordinate(x['station_id'],x['station_lat'],x['station_lon']),axis=1)
climate_yearly

,station_id,station_lat,station_lon,elevation,station_name,year,mean_temp,min_temp,max_temp,dewpoint,...,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado,ID_Coordinate
0,10000199999,51.602414,6.142172,32.30,"NIEDERRHEIN, GM",2020,54.705725,47.703448,60.793103,45.812214,...,5.816412,7.288550,0.000000,15,136,2,0,4,0,"[10000199999, 51.602414, 6.142172]"
1,1001099999,70.933333,-8.666667,9.00,"JAN MAYEN NOR NAVY, NO",2020,32.702466,29.169589,36.341370,28.609041,...,9.962117,14.442192,0.092577,111,173,143,2,0,0,"[1001099999, 70.9333333, -8.6666667]"
2,1001499999,59.791925,5.340850,48.76,"SORSTOKKEN, NO",2020,49.737367,46.426277,52.768727,43.220641,...,5.941993,8.966904,0.000000,10,159,22,1,1,0,"[1001499999, 59.791925, 5.34085]"
3,10018099999,54.913250,8.340472,15.54,"WESTERLAND SYLT, GM",2020,52.437152,48.524116,56.634615,47.586131,...,5.510526,12.553251,0.000000,8,110,0,0,2,0,"[10018099999, 54.91325, 8.340472]"
4,1002099999,80.050000,16.250000,8.00,"VERLEGENHUKEN, NO",2020,17.036948,12.869880,21.083936,13.459438,...,NaN,12.864659,0.000000,0,0,0,0,0,0,"[1002099999, 80.05, 16.25]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12255,A0735500241,43.579000,-90.913000,394.10,"VIROQUA MUNICIPAL AIRPORT, WI US",2020,46.622527,38.970604,56.151648,40.098901,...,9.184890,6.720055,0.000000,0,0,0,0,0,0,"[A0735500241, 43.579, -90.913]"
12256,A0735700182,45.986000,-95.992000,367.30,ELBOW LAKE MUNICIPAL PRIDE OF THE PRAIRIE AIRP...,2020,41.462170,32.972434,51.273021,34.003226,...,9.381232,8.385044,0.000000,23,78,70,0,21,0,"[A0735700182, 45.986, -95.992]"
12257,A0735900240,42.938000,-85.061000,249.00,"IONIA CO AIRPORT, MI US",2020,49.277473,41.189011,58.412088,37.301374,...,9.413187,7.039011,0.000000,17,135,60,0,0,0,"[A0735900240, 42.938, -85.061]"
12258,A5125500445,32.463830,-87.954050,34.10,"DEMOPOLIS MUNICIPAL AIRPORT, AL US",2020,65.483989,56.372472,76.796067,57.367697,...,9.095787,2.741011,0.000000,29,162,4,0,0,0,"[A5125500445, 32.46383, -87.95405]"


In [42]:
pointList = climate_yearly['ID_Coordinate'].tolist()
pointList[:20]

[['10000199999', 51.602414, 6.142172],
 ['1001099999', 70.9333333, -8.6666667],
 ['1001499999', 59.791925, 5.34085],
 ['10018099999', 54.91325, 8.340472],
 ['1002099999', 80.05, 16.25],
 ['1003099999', 77.0, 15.5],
 ['10033099999', 54.8333333, 9.5],
 ['10037099999', 54.459333, 9.516333],
 ['10038099999', 54.312167, 9.538167],
 ['10046599999', 54.3795, 10.145167],
 ['1006099999', 78.25, 22.8166666],
 ['1007099999', 78.9166666, 11.9333333],
 ['1008099999', 78.246111, 15.465556],
 ['1009099999', 80.65, 25.0],
 ['1010099999', 69.2925, 16.144167],
 ['1011099999', 80.0666666, 31.5],
 ['10126099999', 53.547833, 7.667333],
 ['10136099999', 53.767667, 8.6585],
 ['10147099999', 53.630389, 9.988228],
 ['10149099999', 53.535886, 9.837025]]

### create functions to find the nearest weather station for each county

In [43]:
def dist(p0, p1):
    return (((p0[0] - p1[0])**2) + ((p0[1] - p1[1])**2))**.5

#### We don't need to create a list of distance with all the weather stations. 
#### We can find the square around the county centroid and the square should contain a list of weather stations. 
#### Then we can find the nearest one

In [44]:
def findSquare(pointList, lat, lon):
    square = []
    for point in pointList:
        point_lat = point[1]
        point_lon = point[2]
        if ((point_lat < lat + 1)    # I chosed to make the square with 1 longitude and 1 latitude away from the centroid
            and (point_lat > lat - 1)
            and (point_lon < lon + 1)
            and (point_lon > lon - 1)):
            square.append(point)
    return square

In [45]:
### This is the main function used to find the nearest weather station
def shortestDistance(lat, lon):
    p1 = []
    p1.append(lat)
    p1.append(lon)
    distants = []
    square = findSquare(pointList, lat, lon)
    for point in square:
        p_id = point[0]
        p_lat = point[1]
        p_lon = point[2]
        p2 = []
        p2.append(p_lat)
        p2.append(p_lon)
        distant = dist(p1, p2)
        dist_id = []
        dist_id.append(p_id)
        dist_id.append(distant)
        distants.append(dist_id)
    distantList = []    
    for ele in distants:
        distantList.append(ele[1])
    for item in distants:
        if item[1] == min(distantList):
            return item[0]

In [46]:
county['NearestStation'] = county.apply(lambda x: shortestDistance(x['lat'],x['lon']),axis=1)
county

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,Centroid,lat,lon,NearestStation
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",POINT (-88.99925588833919 37.05848196169251),37.058482,-88.999256,72497593809
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",POINT (-84.21715057392247 38.20673452494952),38.206735,-84.217151,72045600146
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",POINT (-86.68162334315123 37.20728450321774),37.207285,-86.681623,74671693808
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",POINT (-83.96431121402227 37.69244422727429),37.692444,-83.964311,72045500145
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",POINT (-83.69665601093851 38.37011818128263),38.370118,-83.696656,72045299999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,31,073,835858,0500000US31073,31073,Gosper,6,1186616237,11831826,"POLYGON ((-100.095095 40.438656, -100.089374 4...",POINT (-99.83070272142194 40.51481873658771),40.514819,-99.830703,72562414994
3229,39,075,1074050,0500000US39075,39075,Holmes,6,1094405866,3695230,"POLYGON ((-82.220665 40.66758, -82.193271 40.6...",POINT (-81.92934087401707 40.56120787095739),40.561208,-81.929341,99999954856
3230,48,171,1383871,0500000US48171,48171,Gillespie,6,2740719114,9012764,"POLYGON ((-99.303996 30.499831999999998, -99.2...",POINT (-98.94655484362227 30.31803705160264),30.318037,-98.946555,72032393947
3231,55,079,1581100,0500000US55079,55079,Milwaukee,6,625440563,2455383635,"POLYGON ((-88.069586 42.867263, -88.069586 42....",POINT (-87.96677179509463 43.00719656824836),43.007197,-87.966772,72640014839


In [48]:
# rename the 'NearestStation' to 'station_id' and merge with the climate 
county = county.rename(columns = {'NearestStation': 'station_id'})
county_climate = county.merge(climate_yearly, on ='station_id', how ='left')
county_climate.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado,ID_Coordinate
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",...,9.032240,5.140260,0.075452,0.0,7.0,0.0,0.0,3.0,0.0,"[72497593809, 37.06444, -89.21944]"
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",...,9.468767,4.831781,0.000000,13.0,141.0,25.0,0.0,0.0,0.0,"[72045600146, 38.067, -83.983]"
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",...,9.323224,4.733060,0.155628,27.0,159.0,9.0,0.0,44.0,0.0,"[74671693808, 36.9647, -86.4238]"
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",...,9.224044,4.856011,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,"[72045500145, 37.633, -84.333]"
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",...,9.315847,5.749454,0.000000,29.0,171.0,19.0,0.0,0.0,0.0,"[72045299999, 38.541806, -83.743389]"


In [49]:
county_climate['fips'] = county_climate['STATEFP'] + county_climate['COUNTYFP']
county_climate.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado,ID_Coordinate,fips
0,21,007,516850,0500000US21007,21007,Ballard,6,639387454,69473325,"POLYGON ((-89.18136899999999 37.046305, -89.17...",...,5.140260,0.075452,0.0,7.0,0.0,0.0,3.0,0.0,"[72497593809, 37.06444, -89.21944]",21007
1,21,017,516855,0500000US21017,21017,Bourbon,6,750439351,4829777,"POLYGON ((-84.442661 38.283235999999995, -84.4...",...,4.831781,0.000000,13.0,141.0,25.0,0.0,0.0,0.0,"[72045600146, 38.067, -83.983]",21017
2,21,031,516862,0500000US21031,21031,Butler,6,1103571974,13943044,"POLYGON ((-86.944858 37.073406999999996, -86.9...",...,4.733060,0.155628,27.0,159.0,9.0,0.0,44.0,0.0,"[74671693808, 36.9647, -86.4238]",21031
3,21,065,516879,0500000US21065,21065,Estill,6,655509930,6516335,"POLYGON ((-84.126618 37.645404, -84.1248339999...",...,4.856011,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,"[72045500145, 37.633, -84.333]",21065
4,21,069,516881,0500000US21069,21069,Fleming,6,902727151,7182793,"POLYGON ((-83.984282 38.445493, -83.98246 38.4...",...,5.749454,0.000000,29.0,171.0,19.0,0.0,0.0,0.0,"[72045299999, 38.541806, -83.743389]",21069


In [50]:
fips_station = county_climate[['fips', 'station_id']]
fips_station

,fips,station_id
0,21007,72497593809
1,21017,72045600146
2,21031,74671693808
3,21065,72045500145
4,21069,72045299999
...,...,...
3228,31073,72562414994
3229,39075,99999954856
3230,48171,72032393947
3231,55079,72640014839


### save the yearly county level climate dataframe 

In [51]:
climate_county_yearly = county_climate[['fips', 'year','mean_temp', 'min_temp','max_temp', 'dewpoint', 
                                        'sea_level_pressure', 'station_pressure','visibility', 'wind_speed',
                                        'precipitation', 'fog', 'rain', 'snow','hail', 'thunder','tornado']]
climate_county_yearly

,fips,year,mean_temp,min_temp,max_temp,dewpoint,sea_level_pressure,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,21007,2020.0,57.769672,49.621858,67.730601,49.837158,NaN,138.545730,9.032240,5.140260,0.075452,0.0,7.0,0.0,0.0,3.0,0.0
1,21017,2020.0,55.846027,47.578630,66.130959,47.524384,NaN,979.099725,9.468767,4.831781,0.000000,13.0,141.0,25.0,0.0,0.0,0.0
2,21031,2020.0,59.798361,50.946448,70.473497,49.180328,1018.212568,611.321154,9.323224,4.733060,0.155628,27.0,159.0,9.0,0.0,44.0,0.0
3,21065,2020.0,57.415574,48.573770,68.703279,48.636612,NaN,NaN,9.224044,4.856011,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,21069,2020.0,55.419945,47.169126,66.003825,46.666120,NaN,NaN,9.315847,5.749454,0.000000,29.0,171.0,19.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,31073,2020.0,51.190984,39.472678,66.115152,39.554645,NaN,931.032418,9.377869,8.362022,0.039394,23.0,91.0,37.0,0.0,33.0,0.0
3229,39075,2020.0,51.722404,43.935519,61.168306,NaN,NaN,NaN,NaN,NaN,0.111885,0.0,0.0,0.0,0.0,0.0,0.0
3230,48171,2020.0,65.479670,54.856319,78.430769,51.933516,NaN,956.768681,9.528297,4.968956,0.067740,22.0,91.0,1.0,0.0,24.0,0.0
3231,55079,2020.0,50.727596,41.901366,61.127869,39.032240,1016.631694,903.429315,9.325683,8.354098,0.114290,18.0,131.0,57.0,1.0,34.0,0.0


In [53]:
len(climate_county_yearly['fips'].unique())

3233

In [54]:
climate_county_yearly['mean_temp'].isna().sum()

9

In [55]:
climate_county_yearly.to_csv('climate_county_level_yearly/climate_county_level_yearly_2020.csv')

### Save the daily climate dataframe

In [56]:
climate = climate.merge(fips_station, on = 'station_id', how = 'left')
climate.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,min_temp,max_temp,dewpoint,...,precipitation,fog,rain,snow,hail,thunder,tornado,year,Month,fips
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,82.2,83.3,73.2,...,0.00,0,0,0,0,0,0,2020,Mar,NaN
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,82.4,83.7,72.4,...,0.00,0,0,0,0,0,0,2020,Mar,NaN
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23.5,35.8,29.3,...,0.37,0,1,0,0,0,0,2020,Jan,NaN
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,19.0,34.2,23.4,...,0.54,1,0,1,0,0,0,2020,Jan,NaN
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,19.6,33.6,18.9,...,0.31,0,1,0,0,0,0,2020,Jan,NaN


In [57]:
climate

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,min_temp,max_temp,dewpoint,...,precipitation,fog,rain,snow,hail,thunder,tornado,year,Month,fips
0,841599999,2020-03-13,NaN,NaN,NaN,XM21,82.7,82.2,83.3,73.2,...,0.00,0,0,0,0,0,0,2020,Mar,NaN
1,841599999,2020-03-14,NaN,NaN,NaN,XM21,83.1,82.4,83.7,72.4,...,0.00,0,0,0,0,0,0,2020,Mar,NaN
2,1001099999,2020-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23.5,35.8,29.3,...,0.37,0,1,0,0,0,0,2020,Jan,NaN
3,1001099999,2020-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,19.0,34.2,23.4,...,0.54,1,0,1,0,0,0,2020,Jan,NaN
4,1001099999,2020-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",25.1,19.6,33.6,18.9,...,0.31,0,1,0,0,0,0,2020,Jan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4631204,A5125600451,2020-12-29,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",34.9,32.0,39.2,31.4,...,NaN,0,1,1,0,0,0,2020,Dec,29209
4631205,A5125600451,2020-12-30,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",41.5,32.0,51.8,41.5,...,NaN,1,1,1,0,0,0,2020,Dec,05015
4631206,A5125600451,2020-12-30,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",41.5,32.0,51.8,41.5,...,NaN,1,1,1,0,0,0,2020,Dec,29209
4631207,A5125600451,2020-12-31,36.698500,-93.402200,411.2,"BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...",30.7,26.6,35.6,29.5,...,0.00,0,0,0,0,0,0,2020,Dec,05015


In [58]:
len(climate['fips'].unique())

3225

In [59]:
climate_daily = climate[['fips', 'date', 'year', 'Month', 'mean_temp', 'min_temp', 'max_temp', 'dewpoint', 
                         'sea_level_pressure', 'station_pressure','visibility', 'wind_speed','precipitation',
                         'fog', 'rain', 'snow','hail', 'thunder', 'tornado']]
climate_daily

,fips,date,year,Month,mean_temp,min_temp,max_temp,dewpoint,sea_level_pressure,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,NaN,2020-03-13,2020,Mar,82.7,82.2,83.3,73.2,1010.9,NaN,NaN,14.5,0.00,0,0,0,0,0,0
1,NaN,2020-03-14,2020,Mar,83.1,82.4,83.7,72.4,1008.7,NaN,NaN,11.0,0.00,0,0,0,0,0,0
2,NaN,2020-01-01,2020,Jan,31.5,23.5,35.8,29.3,974.9,973.8,3.7,9.9,0.37,0,1,0,0,0,0
3,NaN,2020-01-02,2020,Jan,26.0,19.0,34.2,23.4,960.1,959.0,4.9,23.0,0.54,1,0,1,0,0,0
4,NaN,2020-01-03,2020,Jan,25.1,19.6,33.6,18.9,985.5,984.3,NaN,33.4,0.31,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4631204,29209,2020-12-29,2020,Dec,34.9,32.0,39.2,31.4,NaN,976.6,6.3,8.7,NaN,0,1,1,0,0,0
4631205,05015,2020-12-30,2020,Dec,41.5,32.0,51.8,41.5,NaN,968.2,4.5,6.9,NaN,1,1,1,0,0,0
4631206,29209,2020-12-30,2020,Dec,41.5,32.0,51.8,41.5,NaN,968.2,4.5,6.9,NaN,1,1,1,0,0,0
4631207,05015,2020-12-31,2020,Dec,30.7,26.6,35.6,29.5,NaN,971.7,8.9,5.9,0.00,0,0,0,0,0,0


In [61]:
### we can use groupby function to find the unique record for each fips and each day
climate_daily = climate_daily.groupby(['fips', 'date']).mean()
climate_daily = climate_daily.reset_index()
climate_daily

,fips,date,year,mean_temp,min_temp,max_temp,dewpoint,sea_level_pressure,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,01001,2020-01-01,2020.0,40.7,29.8,57.7,NaN,NaN,NaN,NaN,1.5,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,01001,2020-01-02,2020.0,49.0,43.3,55.4,NaN,NaN,NaN,NaN,4.9,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,01001,2020-01-03,2020.0,62.3,54.3,69.4,NaN,NaN,NaN,NaN,4.7,1.39,0.0,0.0,0.0,0.0,0.0,0.0
3,01001,2020-01-04,2020.0,55.6,46.8,59.9,NaN,NaN,NaN,NaN,5.0,0.24,0.0,0.0,0.0,0.0,0.0,0.0
4,01001,2020-01-05,2020.0,41.5,28.4,57.4,NaN,NaN,NaN,NaN,2.8,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146954,78030,2020-12-27,2020.0,77.4,74.1,81.5,NaN,1012.6,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1146955,78030,2020-12-28,2020.0,77.6,74.7,81.0,NaN,1013.6,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1146956,78030,2020-12-29,2020.0,78.2,73.4,80.8,NaN,1013.0,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1146957,78030,2020-12-30,2020.0,79.0,78.1,79.7,NaN,1014.1,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
climate_daily['dewpoint'].isna().sum()

90092

In [65]:
na_rate = 90092/1146959
na_rate

0.07854857933021145

In [66]:
climate_daily.to_csv('climate_county_level_daily/climate_county_level_daily_2020.csv')